# Project Operations
Most Literary operations act upon the *project* - that is, the set of notebooks which are considered "packages". Here we define a `Configurable` which defines the configuration that refers to these notebooks

In [1]:
%load_ext literary.notebook

In [2]:
from pathlib import Path

from traitlets import List, Unicode, default, observe, validate
from traitlets.config import Config, LoggingConfigurable

from .trait import Path as PathTrait

In [3]:
class ProjectOperator(LoggingConfigurable):
    project_path = PathTrait(help="Path to Literary project top-level directory").tag(
        config=True
    )

    packages_dir = Unicode(
        "src", help="Path to Literary packages top-level directory"
    ).tag(config=True)

The `project_path` should point to the directory containing the Literary configuration file. We need this directory to exist for subsequent path resolution.

In [4]:
@patch(ProjectOperator)
@validate("project_path")
def _validate_project_path(self, proposal):
    path = Path(proposal.value)
    if not path.exists():
        raise ValueError()
    return path

Here we define the function to resolve a path against the project.

In [5]:
@patch(ProjectOperator)
def resolve_path(self, path):
    return self.project_path / path

The `packages_dir` should always be used after it has been resolved, so let's define a getter.

In [6]:
@patch(ProjectOperator)
@property
def packages_path(self) -> Path:
    return self.resolve_path(self.packages_dir)